# generating a large random network topology

In [12]:
using Random
# naive approach: 
function gentopo(n,k,seed)
    p = k/(n-1)
    rng = MersenneTwister(seed);
    rand(rng,n,n) .< p
end

gentopo (generic function with 1 method)

In [13]:
n = 50 # number of neurons
k = 10 # number of synapses per neuron
seed = 1
A = gentopo(n,k,seed)

50×50 BitArray{2}:
 false  false   true  false  false  …   true  false   true  false   true
 false  false  false  false  false      true  false  false  false  false
 false  false  false   true  false     false  false  false  false   true
  true  false  false  false  false     false   true  false   true  false
 false   true   true  false  false     false  false  false   true   true
 false  false  false  false  false  …  false  false  false   true  false
 false   true  false   true   true     false   true   true  false  false
 false  false  false  false  false     false  false   true   true  false
 false  false  false  false   true     false  false  false  false   true
 false  false  false  false  false     false  false   true   true  false
 false  false  false  false  false  …  false  false   true  false  false
 false  false  false   true  false     false  false  false  false  false
 false  false   true  false  false      true  false  false  false  false
     ⋮                          

In [14]:
j = 17 # example spiking neuron
post = A[:,j]

50-element BitArray{1}:
 false
 false
 false
  true
 false
  true
 false
 false
 false
  true
 false
  true
 false
     ⋮
 false
 false
 false
 false
  true
 false
 false
 false
 false
 false
 false
  true

 ### using sparse matrices

In [21]:
using SparseArrays
# needs less memory (n*k instead of n^2)
function gensparsetopo(n,k,seed)
    rng = MersenneTwister(seed);
    p = k/(n-1)
    A = sprand(Bool,n,n,p)
end

gensparsetopo (generic function with 1 method)

In [22]:
Asparse = gensparsetopo(n,k,seed)

50×50 SparseMatrixCSC{Bool,Int64} with 526 stored entries:
  [1 ,  1]  =  true
  [5 ,  1]  =  true
  [6 ,  1]  =  true
  [8 ,  1]  =  true
  [9 ,  1]  =  true
  [12,  1]  =  true
  [20,  1]  =  true
  [24,  1]  =  true
  [30,  1]  =  true
  [32,  1]  =  true
  [33,  1]  =  true
  [34,  1]  =  true
  ⋮
  [1 , 50]  =  true
  [7 , 50]  =  true
  [11, 50]  =  true
  [14, 50]  =  true
  [15, 50]  =  true
  [18, 50]  =  true
  [25, 50]  =  true
  [27, 50]  =  true
  [28, 50]  =  true
  [36, 50]  =  true
  [41, 50]  =  true
  [43, 50]  =  true

In [23]:
j = 17 # example spiking neuron
post = A[:,j]

50-element BitArray{1}:
 false
 false
 false
  true
 false
  true
 false
 false
 false
  true
 false
  true
 false
     ⋮
 false
 false
 false
 false
  true
 false
 false
 false
 false
 false
 false
  true

### generating network topology on the fly

In [25]:
# no need to store network topology of size n*k
using StatsBase
j = 18
mt_rng = MersenneTwister(j)
post = sample(1:n-1,k) # generate postsynaptic neuron index

10-element Array{Int64,1}:
 15
 10
  6
 23
 15
 39
 31
 48
 26
 18

###### bottleneck: seeding the random number generator

In [28]:
using BenchmarkTools
mt = MersenneTwister(seed)
@btime Random.seed!(mt,1);

  11.327 μs (2 allocations: 112 bytes)


###### Solution: using faster random number generator

In [29]:
using RandomNumbers.Xorshifts
r = Xoroshiro128Star(seed)
@btime Random.seed!(r,1);

  18.501 ns (0 allocations: 0 bytes)
